# Run analysis on multiple files

- The simulations should be in the form of distinct folders containing results for different parameters within the specified folder as follows:

data_folder
   - sim_folder_1
   - sim_folder_2
   - ...

In [5]:
import sys
if 'init_modules' in globals().keys():
    # second or subsequent run: remove all but initially loaded modules
    for m in sys.modules.keys():
        if m not in init_modules:
            del(sys.modules[m])
else:
    # first run: find out which modules were initially loaded
    init_modules = sys.modules.keys()
    
import numpy as np
import pandas as pd
from scipy import signal
from scipy import interpolate
import matplotlib.pyplot as plt 
import os
import pyfilaments.analysisutils as analysis
from tqdm import tqdm

from joblib import Parallel, delayed
import multiprocessing
import h5py

In [10]:
# data_folder = '/home/deepak/ActiveFilamentsSearch_backup_3/BendingStiffnessSweeps/analysis_test'
# data_folder = '/home/deepak/ActiveFilamentsSearch_backup_3/BendingStiffnessSweeps/b0_2_1_FullSweep_Final'

# data_folder = '/home/deepak/ActiveFilamentsSearch_backup_3/FinerResolution_BendingStiffness'
# data_folder = '/home/deepak/ActiveFilamentsSearch_backup_3/ModellingResults/2021-06-23'
# data_folder = '/home/deepak/ActiveFilamentsSearch_backup_3/ModellingResults'

# data_folder = '/home/deepak/ActiveFilamentsSearch_backup_3/ModellingResults/2021-08-18'
# data_folder = '/home/deepak/Dropbox/LacryModeling/ModellingResults/StochasticActivity/NormalDistribution'
# data_folder = '/home/deepak/ActiveFilamentsSearch_backup_3/BendingStiffnessSweeps/b0_2_1_FullSweep_Final/activity_time_750'

# data_folder = '/home/deepak/ActiveFilamentsSearch_backup_3/FilamentLength_Sweeps/activity_time_750'
# data_folder = '/home/deepak/ActiveFilamentsSearch_backup_3/ModellingResults/2021-08-28'

# data_folder = '/home/deepak/ActiveFilamentsSearch_backup_3/ModellingResults/2021-10-19'

# data_folder = '/home/deepak/ActiveFilamentsSearch_backup_3/ActivityStrength_sweep'

# data_folder = '/home/deepak/ActiveFilamentsSearch_backup_3/ModellingResults/2021-11-08'

# data_folder = '/home/deepak/ActiveFilamentsSearch_backup_3/ConstantCompressiveActivity_sweep'

# data_folder = '/home/deepak/ActiveFilamentsSearch_backup_3/ModellingResults/2021-11-25'

# data_folder = '/home/deepak/ActiveFilamentsSearch_backup_3/ModellingResults/2021-12-02'

# Axial stiffness sweep
# data_folder = '/home/deepak/ActiveFilamentsSearch_backup_3/ActivityStrength_sweep_k_10_kappa_6.25'

# data_folder = '/home/deepak/ActiveFilamentsSearch_backup_3/ActivityStrength_sweep_k_100_kappa_6.25'

# data_folder = '/home/deepak/ActiveFilamentsSearch_backup_3/ActivityStrength_sweep_RandomAnglesIC'

data_folder = '/home/deepak/ActiveFilamentsSearch_backup_3/ModellingResults/Analysis_test'

# data_folder = '/home/deepak/ActiveFilamentsSearch_backup_3/ActivityStrengthSweeps_diffActivityTimescales/activity_timescale_700'

# data_folder = '/home/deepak/ActiveFilamentsSearch_backup_3/ActivityTimeSweep_constantActivity_1.5'

# data_folder = '/home/deepak/ActiveFilamentsSearch_backup_3/StochasticActivity/NormalDistribution/noise_0_05'

# data_folder = '/home/deepak/ActiveFilamentsSearch_backup_3/StochasticActivity/DeterministicActivity'

# data_folder = '/home/deepak/ActiveFilamentsSearch_backup_3/ActivityStrengthSweeps_diffActivityTimeScales_FINAL'

# data_folder = '/home/deepak/ActiveFilamentsSearch_backup_3/ActivityStrengthSweep_widerICS_FINAL_2021-12-23'

# data_folder = '/home/deepak/ActiveFilamentsSearch_backup_3/ActivityStrengthSweep_FineResolution'

# data_folder = '/home/deepak/ActiveFilamentsSearch_backup_3/DistributedActivitySims_2022_11_11'

# data_folder = '/home/deepak/ActiveFilamentsSearch_backup_3/ModellingResults/2022-11-12'

# data_folder = '/home/deepak/ActiveFilamentsSearch_backup_3/ActivityStrengthSweeps_diffActivityTimeScales_FINAL/activityTime_500'

# data_folder = '/home/deepak/ActiveFilamentsSearch_backup_3/ActivityTimeSweep_constantActivity_1.5'

# data_folder = '/home/deepak/ActiveFilamentsSearch_backup_3/ActivityTimeSweep_FINAL'

# data_folder = '/home/deepak/ActiveFilamentsSearch_backup_3/DistributedActivity_LacryType/NeckHead_scalefactor_2'

# data_folder = '/home/deepak/ActiveFilamentsSearch_backup_3/DistributedActivitySims_2022_11_11'

# data_folder = '/home/deepak/ActiveFilamentsSearch_backup_3/ModellingResults/2022-12-27'

# Find all simulation data files and create a list
files_list = []
 # Walk through the folders and identify the simulation data files
for dirs, subdirs, files in os.walk(data_folder, topdown=False):

    root, subFolderName = os.path.split(dirs)

    for fileNames in files:
        if(fileNames.endswith('hdf5') and fileNames[0] != '.' and 'analysis' not in fileNames and 'ic' not in fileNames and 'eigenvectors' not in fileNames):
            files_list.append(os.path.join(dirs,fileNames))
#         if(fileNames.endswith('hdf5') and fileNames[0] != '.' and fileNames == 'SimResults_00.hdf5'):
#             files_list.append(os.path.join(dirs,fileNames))

print('No:of Simulation files: ', len(files_list))


    
N_ACTIVITY_CYCLES = 400 # No:of activity cycles requested

No:of Simulation files:  40


In [11]:
def run_filament_analysis(file):
    print('Analyzing file ...')
    print(file)

    filament = analysis.analysisTools(file = file)
    

    root_folder, file_name = os.path.split(file)
    
    # @@@ Better to use Metadata from the HDF5 file so there is no possible delinking of data and metadata
    df = pd.DataFrame({'N particles':[filament.Np],'radius':[filament.radius],'bond length':[filament.b0],
                       'spring constant':[filament.k], 'kappa_hat':[filament.kappa_hat], 
                       'force strength':[filament.F0], 'stresslet strength':[filament.S0], 
                       'potDipole strength':[filament.D0], 'simulation type':[filament.sim_type], 
                       'boundary condition 0':[filament.bc[0]], 'boundary condition 1':[filament.bc[-1]], 
                       ' activity time scale':[filament.activity_timescale], 'viscosity':[filament.mu]})
    
    # Calculate dimensionless numbers
    filament.time_scales()
    filament.compute_scales()
    filament.compute_dimensionless_groups()
    
    periodic_flag = False 
    min_period = None
    threshold_index = int(0)  # Activity cycle when the dynamics transitions to a periodic cycle
    
    
    # Filament tip coverage analysis
    filament.filament_tip_coverage(save = True, overwrite = True)
    
    # Calculate the search efficiency
    search_efficiency = filament.search_efficiency()
    
    print(filament.activity_timescale)
    # Check if simulation was completed or terminated before completion
    if(int(filament.Time[-1]) >= int(filament.activity_timescale*N_ACTIVITY_CYCLES)):
        simulation_completed = True
        # Classify the dynamics
        periodic_flag, min_period, threshold_index = filament.classify_filament_dynamics()
    else:
        simulation_completed = False
        
    
    activity_cycles_completed = int(filament.Time[-1]/filament.activity_timescale)
    
    if(periodic_flag == False):
        # if dynamics is aperiodic
        df['period'] = float(np.nan)
        df['threshold index'] = int(0)
    else:
        df['period'] = min_period
        df['threshold index'] = int(threshold_index)
        
        
    df['simulation completed'] = simulation_completed
    df['periodic dynamics'] = periodic_flag
    
    print('Periodic dynamics:', df['periodic dynamics'])
    print('Period:', df['period'])
    
    df['max unique locations'] = filament.derived_data['unique position count'][-1]
    df['activity cycles completed'] = activity_cycles_completed
    
    print('Activity cycles completed: {}'.format(activity_cycles_completed))
    
    df['search efficiency'] = search_efficiency
    
    df['activity number'] = filament.activity_number
    
#     df['activity type'] =  filament.activity['type']
#     df['noise scale'] =  filament.activity['noise_scale']
    
    filament_buckling = filament.detect_buckling()
    
    print(filament_buckling)
    
    if periodic_flag==True:
        
        if min_period==1 and filament_buckling==False:
            df['filament behavior'] = 'no buckling'
        elif min_period==1 and filament_buckling == True:
            df['filament behavior'] = '1-period'
        elif min_period%2==0:
            df['filament behavior']= '2n-period'
        else:
            df['filament behavior'] = 'odd-period'
    else:
        df['filament behavior'] = 'aperiodic'
        
    if simulation_completed == False:
        df['filament behavior'] = 'escape'
    
    # Calculate the filament tip angles (at constant phase) and save them to a file
    
    phase = 0
    angles = filament.compute_basetip_angle_at_constant_phase(phase_value = phase)
    
#     print(angles)
    
    
    # Save the analysis data
    save_file = file_name[:-5] + '_analysis.csv'
    save_folder = os.path.join(root_folder, 'Analysis')

    if(not os.path.exists(save_folder)):
        os.makedirs(save_folder)
        
    df.to_csv(os.path.join(save_folder, save_file))
    
    # Create a dataset and save the data
    save_file = file_name[:-5] + '_analysis.hdf5'


    with h5py.File(os.path.join(save_folder, save_file), "w") as f:
        dset = f.create_group("analysis data")
        dset.attrs['N particles'] = filament.Np
        dset.attrs['radius'] = filament.radius
        dset.attrs['bond length'] = filament.b0
        dset.attrs['spring constant'] = filament.k
        dset.attrs['kappa_hat'] = filament.kappa_hat
        dset.attrs['force strength'] = filament.F0
        dset.attrs['stresslet strength'] = filament.S0
        dset.attrs['potDipole strength'] = filament.D0
        dset.attrs['simulation type'] = filament.sim_type
        dset.attrs['activity timescale'] = filament.activity_timescale
        dset.attrs['viscosity'] = filament.mu
        dset.attrs['boundary condition 0'] = filament.bc[0]
        dset.attrs['boundary condition 1'] = filament.bc[-1]
        dset.attrs['threshold cycle'] = df['threshold index']  # This is the cause of the issue...
        
        print(periodic_flag)
        print(min_period)
        dset.attrs['activity cycles completed'] = activity_cycles_completed

        dset.attrs['Sim complete'] = int(simulation_completed)

        dset.attrs['periodic'] = df['periodic dynamics']
        dset.attrs['period'] =  df['period']
        
        dset.attrs['constant phase'] = phase
        dset.create_dataset("Tip angles", data = angles)
        
        dset.attrs['activity number'] = filament.activity_number
        dset.attrs['filament behavior'] = df['filament behavior'][0]

In [12]:
num_cores = multiprocessing.cpu_count()

num_cores = 12

results = Parallel(n_jobs=num_cores,  verbose=10)(delayed(run_filament_analysis)(file) for file in tqdm(files_list))


100%|██████████| 40/40 [00:11<00:00,  3.53it/s]
[Parallel(n_jobs=12)]: Done  17 tasks      | elapsed:   13.4s
[Parallel(n_jobs=12)]: Done  22 out of  40 | elapsed:   15.2s remaining:   12.4s
[Parallel(n_jobs=12)]: Done  27 out of  40 | elapsed:   16.2s remaining:    7.8s
[Parallel(n_jobs=12)]: Done  32 out of  40 | elapsed:   18.1s remaining:    4.5s
[Parallel(n_jobs=12)]: Done  37 out of  40 | elapsed:   21.0s remaining:    1.7s
[Parallel(n_jobs=12)]: Done  40 out of  40 | elapsed:   40.5s finished
